In [2]:
%load_ext autoreload
%autoreload 2

from helpers import *

from surprise import Dataset
from surprise import Reader

from surprise.model_selection import GridSearchCV

path_dataset = "data/data_train.csv"
data = load_data(path_dataset)
dataset = Dataset.load_from_df(data[['user', 'movie', 'rating']], Reader(rating_scale=(1, 5)))

data.head() 

,user,movie,rating
0,r44,c1,4
1,r61,c1,3
2,r67,c1,4
3,r72,c1,3
4,r86,c1,5


![alt text](ALS.png "Title")

In [4]:
from ALS_implementation import ALS
param_grid = {"n_epochs": [25], "num_features": [8], "lambda_all": [0.081]}
gs = GridSearchCV(ALS, param_grid, measures=['rmse', 'mae'], cv=6, n_jobs=-1, joblib_verbose=1000) 
gs.fit(dataset)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  9.4min
[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:  9.5min remaining: 19.1min
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:  9.9min remaining:  9.9min
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed: 10.1min remaining:  5.0min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 10.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 10.6min finished


In [5]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
print(gs.cv_results)

0.9875117172802444
{'n_epochs': 25, 'num_features': 8, 'lambda_all': 0.081}
{'split0_test_rmse': array([0.98969969]), 'split1_test_rmse': array([0.98719661]), 'split2_test_rmse': array([0.98633981]), 'split3_test_rmse': array([0.98501662]), 'split4_test_rmse': array([0.98869792]), 'split5_test_rmse': array([0.98811965]), 'mean_test_rmse': array([0.98751172]), 'std_test_rmse': array([0.00154176]), 'rank_test_rmse': array([1], dtype=int64), 'split0_test_mae': array([0.80153347]), 'split1_test_mae': array([0.79900999]), 'split2_test_mae': array([0.79900274]), 'split3_test_mae': array([0.79663436]), 'split4_test_mae': array([0.80007224]), 'split5_test_mae': array([0.80038579]), 'mean_test_mae': array([0.79943976]), 'std_test_mae': array([0.00152341]), 'rank_test_mae': array([1], dtype=int64), 'mean_fit_time': array([547.56412089]), 'std_fit_time': array([2.53279099]), 'mean_test_time': array([1.15858813]), 'std_test_time': array([0.1502079]), 'params': [{'n_epochs': 25, 'num_features': 8, 

In [6]:
submission = pd.read_csv("data/sample_submission.csv")
trainset = dataset.build_full_trainset()
gsbf = gs.best_estimator['rmse'].fit(trainset)

number of items in preprocess_trainset_data: 1000, number of users: 10000

start the ALS algorithm...
RMSE on training set: 1.0279557692829417.
RMSE on training set: 1.002643700880776.
RMSE on training set: 0.984104400950518.
RMSE on training set: 0.9725554764695716.
RMSE on training set: 0.9649053241359405.
RMSE on training set: 0.9593962705374679.
RMSE on training set: 0.9554039026823504.
RMSE on training set: 0.952536228342509.
RMSE on training set: 0.9504453053662352.
RMSE on training set: 0.948866722777238.
RMSE on training set: 0.9476306025435011.
RMSE on training set: 0.946633145835352.
RMSE on training set: 0.9458099253295328.
RMSE on training set: 0.945119423907407.
RMSE on training set: 0.9445336891663806.
RMSE on training set: 0.9440329854033788.
RMSE on training set: 0.9436026810899607.
RMSE on training set: 0.9432314284306135.
RMSE on training set: 0.9429101013482459.
RMSE on training set: 0.9426311748098132.
RMSE on training set: 0.9423883539770682.
RMSE on training set: 

In [7]:
def predict(user, movie):
    return int(round(recommenders[cluster_dict[movie]].predict(user, movie).est))

In [8]:
submission['Prediction'] = [int(round(gsbf.predict(user, movie).est)) for [user, movie] in submission['Id'].str.split('_')]
submission.to_csv(r'data/submission_als_adri_1.csv', index=False)